Be sure to change runtime type to GPU if on Google Colab

In [ ]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/content/content/MyDrive/AI & Deep Learning Project/Garbage classification/test/cardboard/cardboard102.jpg')
img_array = image.img_to_array(test_image)

In [ ]:
img_array.shape

(384, 512, 3)

In [ ]:
n_classes = len(os.listdir('/content/content/MyDrive/AI & Deep Learning Project/images'))
n_classes

6

In [ ]:
img_array = image.img_to_array(test_image)
keras.applications.xception.preprocess_input(img_array).shape

(384, 512, 3)

In [ ]:
TEST_PATH = '/content/content/MyDrive/AI & Deep Learning Project/Garbage classification/test'
TRAIN_PATH = '/content/content/MyDrive/AI & Deep Learning Project/Garbage classification/train'

X_valid_list = []
y_valid_list = []

X_train_list = []
y_train_list = []

for folder in os.listdir(TEST_PATH):
  FOLDER_PATH = os.path.join(TEST_PATH, folder)
  for im in os.listdir(FOLDER_PATH):
    img = image.load_img(os.path.join(TEST_PATH, folder, im), target_size=(224, 224))
    img_array = image.img_to_array(img)
    processed_img = keras.applications.xception.preprocess_input(img_array)
    X_valid_list.append(list(processed_img))
    y_valid_list += [folder] # label of the image being passed in
    if len(y_valid_list)%50 == 0:
      print(len(y_valid_list), 'images loaded into test')

for folder in os.listdir(TRAIN_PATH):
  FOLDER_PATH = os.path.join(TRAIN_PATH, folder)
  for im in os.listdir(FOLDER_PATH):
    img = image.load_img(os.path.join(TRAIN_PATH, folder, im), target_size=(224, 224))
    img_array = image.img_to_array(img)
    processed_img = keras.applications.xception.preprocess_input(img_array)
    X_train_list.append(list(processed_img))
    y_train_list += [folder] # label of the image being passed in
    if len(y_train_list)%50 == 0:
      print(len(y_train_list), 'images loaded into train')

50 images loaded into test
100 images loaded into test
150 images loaded into test
200 images loaded into test
250 images loaded into test
300 images loaded into test
350 images loaded into test
400 images loaded into test
450 images loaded into test
500 images loaded into test
50 images loaded into train
100 images loaded into train
150 images loaded into train
200 images loaded into train
250 images loaded into train
300 images loaded into train
350 images loaded into train
400 images loaded into train
450 images loaded into train
500 images loaded into train
550 images loaded into train
600 images loaded into train
650 images loaded into train
700 images loaded into train
750 images loaded into train
800 images loaded into train
850 images loaded into train
900 images loaded into train
950 images loaded into train
1000 images loaded into train
1050 images loaded into train
1100 images loaded into train
1150 images loaded into train
1200 images loaded into train
1250 images loaded in

In [ ]:
X_valid = np.array(X_valid_list)
X_train_full = np.array(X_train_list)
y_valid = pd.factorize(y_valid_list)[0].reshape(-1, 1)
y_train_full = pd.factorize(y_train_list)[0].reshape(-1, 1)

In [ ]:
valid_shuffler = np.random.permutation(len(X_valid))
X_valid = X_valid[valid_shuffler]
y_valid = y_valid[valid_shuffler]

train_full_shuffler = np.random.permutation(len(X_train_full))
X_train_full = X_train_full[train_full_shuffler]
y_train_full = y_train_full[train_full_shuffler]

In [ ]:
test_size = int(len(y_valid)/2)
X_train, X_test = X_train_full[:-test_size], X_train_full[-test_size:]
y_train, y_test = y_train_full[:-test_size], y_train_full[-test_size:]

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(6, activation='softmax')) # prediction layer

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# implementing a custom callback to stop training if we reach accuracy >0.999 because that indicates overfitting on this relatively small dataset
class OverfittingCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy') > 0.999:
      self.model.stop_training = True
      print('Trying to prevent overfitting - stopping training')

In [ ]:
history = model.fit(X_train, y_train, epochs=40, validation_data = (X_valid, y_valid), callbacks = [keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy'), OverfittingCallback()])

Epoch 1/40
55/55 [==============================] - 54s 416ms/step - loss: 0.8199 - accuracy: 0.7400 - val_loss: 0.5126 - val_accuracy: 0.8185
Epoch 2/40
55/55 [==============================] - 20s 365ms/step - loss: 0.3104 - accuracy: 0.8977 - val_loss: 0.4462 - val_accuracy: 0.8533
Epoch 3/40
55/55 [==============================] - 20s 365ms/step - loss: 0.1871 - accuracy: 0.9423 - val_loss: 0.4364 - val_accuracy: 0.8610
Epoch 4/40
55/55 [==============================] - 20s 368ms/step - loss: 0.1122 - accuracy: 0.9709 - val_loss: 0.4017 - val_accuracy: 0.8726
Epoch 5/40
55/55 [==============================] - 20s 369ms/step - loss: 0.0741 - accuracy: 0.9851 - val_loss: 0.4236 - val_accuracy: 0.8591
Epoch 6/40
55/55 [==============================] - 20s 369ms/step - loss: 0.0608 - accuracy: 0.9840 - val_loss: 0.5325 - val_accuracy: 0.8320
Epoch 7/40
55/55 [==============================] - 20s 369ms/step - loss: 0.0565 - accuracy: 0.9874 - val_loss: 0.4637 - val_accuracy: 0.8494

In [ ]:
base_model.trainable = True
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [ ]:
history = model.fit(X_train, y_train, epochs=40, validation_data=(X_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=3, monitor='val_accuracy'), OverfittingCallback()])

Epoch 1/40
55/55 [==============================] - 20s 369ms/step - loss: 0.0488 - accuracy: 0.9897 - val_loss: 0.4574 - val_accuracy: 0.8571
Epoch 2/40
55/55 [==============================] - 20s 368ms/step - loss: 0.0336 - accuracy: 0.9954 - val_loss: 0.4602 - val_accuracy: 0.8629
Epoch 3/40
55/55 [==============================] - 20s 368ms/step - loss: 0.0279 - accuracy: 0.9949 - val_loss: 0.5612 - val_accuracy: 0.8398
Epoch 4/40
55/55 [==============================] - 20s 369ms/step - loss: 0.0257 - accuracy: 0.9949 - val_loss: 0.5469 - val_accuracy: 0.8398
Epoch 5/40
55/55 [==============================] - 20s 369ms/step - loss: 0.0243 - accuracy: 0.9954 - val_loss: 0.5381 - val_accuracy: 0.8417


In [ ]:
incorrect = dict()
for idx, predictions in enumerate(model.predict(X_test)):
  if np.argmax(predictions) != y_test[idx][0]:
    incorrect[idx] = {'Predicted': np.argmax(predictions), 'Actual': y_test[idx][0]}

incorrect_preds = pd.DataFrame.from_dict(incorrect, orient='index')

In [ ]:
incorrect_preds.reset_index()

,index,Predicted,Actual
0,6,4,0
1,7,4,2
2,10,5,4
3,11,3,0
4,19,4,1
5,22,1,0
6,54,4,5
7,59,0,1
8,66,4,5
9,67,3,0


### Model Deployment

In [ ]:
PROJECT_ID = 'ai-project-315815'
BUCKET_NAME = 'ai-project-315815-bucket'

In [ ]:
!gcloud config set project $PROJECT_ID
!echo $PROJECT_ID

Updated property [core/project].
ai-project-315815


In [ ]:
import sys
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
!gsutil mb -p $PROJECT_ID gs://$BUCKET_NAME
!gsutil ls -al gs://$BUCKET_NAME

Creating gs://ai-project-315815-bucket/...


In [ ]:
JOB_DIR = 'gs://' + BUCKET_NAME + '/aiprojectmodels'
export_path = tf.keras.models.save_model(model, JOB_DIR + '/keras_export')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: gs://ai-project-315815-bucket/aiprojectmodels/keras_export/assets


In [ ]:
MODEL_NAME = 'garbage_classifier_low_augmentation'
!gcloud ai-platform models create $MODEL_NAME

Please specify a region:
(For the global endpoint the region needs to be specified as 
'global'.)
 [1] global
 [2] asia-east1
 [3] asia-northeast1
 [4] asia-southeast1
 [5] australia-southeast1
 [6] europe-west1
 [7] europe-west2
 [8] europe-west3
 [9] europe-west4
 [10] northamerica-northeast1
 [11] us-central1
 [12] us-east1
 [13] us-east4
 [14] us-west1
 [15] cancel
Please enter your numeric choice:  1

To make this the default region, run `gcloud config set ai_platform/region global`.


Learn more about regional endpoints and see a list of available regions: https://cloud.google.com/ai-platform/prediction/docs/regional-endpoints
Using endpoint [https://ml.googleapis.com/]
API [ml.googleapis.com] not enabled on project [755436911006]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  y

Enabling service [ml.googleapis.com] on project [755436911006]...
Operation "operations/acf.p2-755436911006-857d1b90-8923-452b-bd07-c2b4f9499869" finished successfully.
Crea

In [ ]:
MODEL_VERSION = 'v1'
KERAS_EXPORT_DIRS = ! gsutil ls $JOB_DIR/keras_export/
SAVED_MODEL_PATH = KERAS_EXPORT_DIRS[0]

! gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --framework tensorflow \
  --origin $SAVED_MODEL_PATH

Please specify a region:
(For the global endpoint the region needs to be specified as 
'global'.)
 [1] global
 [2] asia-east1
 [3] asia-northeast1
 [4] asia-southeast1
 [5] australia-southeast1
 [6] europe-west1
 [7] europe-west2
 [8] europe-west3
 [9] europe-west4
 [10] northamerica-northeast1
 [11] us-central1
 [12] us-east1
 [13] us-east4
 [14] us-west1
 [15] cancel
Please enter your numeric choice:  1

To make this the default region, run `gcloud config set ai_platform/region global`.

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name
